# EEG - Flow

## 1. Convert XDF to FIFF

The Lab Streaming Layer streams (LSL) stored in the *.xdf* file must be loaded and converted into a FIFF format.
The *.xdf* file contains:
- *eegoSports 000xxx*: the EEG data stream, with `000xxx` the serial number (S/N) of the amplifier.
- *Oddball_task*: triggers of the oddball task, duplicate of the hardware triggers from received on the `TRIGGER` channel of the EEG data stream.
- *OBS_webcam* (task-UT-only): Frame IDs from the webcam OBS video stream.
- *OBS_gameplay* (task-UT-only): Frame IDs from the screen capture OBS video stream.
- *UT_GameEvents* (task-UT-only): Events from Unreal Tournament.
- *MouseButtons* (task-UT-only): Mouse clicks and button events.
- *MousePosition* (task-UT-only): Mouse X/Y position.
- *Keyboard* (task-UT-only): Keyboard button events.

Last edit: 11.05.2023 14:24 (Mathieu)

In [ ]:
import multiprocessing as mp
import time
from itertools import product
from typing import List

from tqdm import tqdm

from eeg_flow.tasks import convert_xdf_to_fiff

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
participants_with_group: List[str] = []  # List of str "Pxx-Gy", e.g. ["P02-G2"]
tasks: List[str] = []  # ["oddball"], ["UT"] or ["oddball", "UT"]
runs: List[int] = []  # [1], [2] or [1, 2]

inputs = [(t[0].split('-') + list(t[1:])) for t in product(participants_with_group, tasks, runs)]
print(inputs)

The variable `inputs` contains is a list of list. Each sublist defines one file by its participant, group, task and run attribute. Each sublist is one set of input variable for `convert_xdf_to_fiff` which will be picked up by a worker (process) and executed. For each execution, the created derivatives are:
- RAW recording (`_raw.fif`)
- Oddball task annotations (`_oddball_annot.fif`)
- Additional stream annotations (keyboard, mouse, .. only if task is `UT`, `_stream_annot.fif`)

In [ ]:
%%time
current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

n_processes = min(len(inputs), mp.cpu_count() - 2)
with mp.Pool(processes=n_processes, maxtasksperchild=10) as p:
    # the progressbar is updated when a process picks-up a new input.
    p.starmap(convert_xdf_to_fiff, tqdm(inputs, total=len(inputs)))